In [2]:

import math, re, ccxt
from pprint import pformat

def get_data_for_graph(price1 = "last", price2 = "last"):
    print('data for: ', price1, price2)
    wex = ccxt.wex()
    data_for_graph = {}
    data = wex.fetch_tickers()
    for key in data:
        data_for_graph.update({key: data[key][price1]})
    ticker_list = list(data_for_graph.keys())
    ticker_list2 = []
    for key in ticker_list:
        ticker_list2.append("{1}/{0}".format(*key.split('/'))) #обратные тикеры
    
    for key in range(0, len(ticker_list)):
        #обратные котировки
        data_for_graph.update({ticker_list2[key]: 1 / (data[ticker_list[key]][price2])})
    
    print(pformat(data_for_graph))
    return data_for_graph
  


In [3]:
'''

Создаем граф в виде списка смежности для каждой ноды - тикера криптовалюты, 
котировки переводим в логарифмический вид, чтобы можо было складывать вместо 
умножения
'''


'\n\nСоздаем граф в виде списка смежности для каждой ноды - тикера криптовалюты, \nкотировки переводим в логарифмический вид, чтобы можо было складывать вместо \nумножения\n'

In [4]:
def create_graph(data_for_graph):
    graph = {}
    jsrates = data_for_graph
    
    pattern = re.compile("([A-Z]{3,5})/([A-Z]{3,5})")
    
    try:
        for key in jsrates:
            matches = pattern.match(key)
    
            conversion_rate = math.log(float(jsrates[key]))
            
            from_rate = matches.group(1).encode('ascii', 'ignore')
    
            to_rate = matches.group(2).encode('ascii', 'ignore')
    
            if from_rate != to_rate:
                if from_rate not in graph:
                    graph[from_rate] = {}
                graph[from_rate][to_rate] = float(conversion_rate)
    
    except AttributeError:
        print('Strange rate')
        pass
    
    #print(pformat(graph))
    # воплощение алгоритма отсюда
    #  https://gist.github.com/joninvski/701720
    return graph


In [5]:

'''
Подготавливаем данные по каждой вершине графа 
и ищем отрицательный цикл в полученном графе

'''

'\nПодготавливаем данные по каждой вершине графа \nи ищем отрицательный цикл в полученном графе\n\n'

In [6]:


# Step 1: For each node prepare the destination and predecessor
def initialize(graph, source):
    d = {}  # Stands for destination
    p = {}  # Stands for predecessor
    for node in graph:
        d[node] = float('Inf')  # We start admiting that the rest of nodes are very very far
        p[node] = None
    d[source] = 0  # For the source we know how to reach
    return d, p


def relax(node, neighbour, graph, d, p):
    # If the distance between the node and the neighbour is lower than the one I have now
    if d[neighbour] > d[node] + graph[node][neighbour]:
        # Record this lower distance
        d[neighbour] = d[node] + graph[node][neighbour]
        p[neighbour] = node


def retrace_negative_loop(p, start):
    arbitrageLoop = [start]
    next_node = start
    while True:
        next_node = p[next_node]
        if next_node not in arbitrageLoop:
            arbitrageLoop.append(next_node)
        else:
            arbitrageLoop.append(next_node)
            arbitrageLoop = arbitrageLoop[arbitrageLoop.index(next_node):]
            return arbitrageLoop


def bellman_ford(graph, source):
    d, p = initialize(graph, source)
    for i in range(len(graph) - 1):  # Run this until is converges
        for u in graph:
            for v in graph[u]:  # For each neighbour of u
                relax(u, v, graph, d, p)  # Lets relax it

    # Step 3: check for negative-weight cycles
    for u in graph:
        for v in graph[u]:
            if d[v] < d[u] + graph[u][v]:
                return (retrace_negative_loop(p, source))
    return None


In [7]:

'''
Применяем алгоритм поиска и выводим ноды, 
входящие в отрицательный цикл, если он существует.
Расчитываем его стоимость с учетом 0.2% комиссии
'''

In [10]:
paths = []

data_for_graph = get_data_for_graph()
graph = create_graph(data_for_graph)
for key in graph:

    path = bellman_ford(graph, key)

    if path not in paths and not None and len(path) > 3:
        paths.append(path)

for path in paths:
    if path == None:
        print("No opportunity here :(")
    else:
        print('cycle lenght:', len(path))
        money = 100
        print("Starting with %(money)i in %(currency)s" % {"money": money, "currency": path[0]})           

        for i, value in enumerate(path):
            if i + 1 < len(path):
                start = path[i]
                end = path[i + 1]
                rate = math.exp(graph[start][end])
                money *= rate*0.998
                print("%(start)s to %(end)s at %(rate)f = %(money)f" % {"start": start, "end": end, "rate": rate,
                                                                        "money": money})
print ("\n")

 

data for:  last last


{'BCH/BCHET': 1.3054830287206267,
 'BCH/BTC': 0.0927,
 'BCH/DASH': 2.123,
 'BCH/ETH': 1.664,
 'BCH/EUR': 541.088,
 'BCH/LTC': 5.514,
 'BCH/RUR': 37268.111,
 'BCH/USD': 655.875,
 'BCH/ZEC': 3.582,
 'BCHET/BCH': 0.766,
 'BTC/BCH': 10.787486515641856,
 'BTC/BTCET': 1.3869625520110958,
 'BTC/DASH': 22.977941176470587,
 'BTC/ETH': 17.924359204158453,
 'BTC/EUR': 5826.78474,
 'BTC/LTC': 59.488399762046406,
 'BTC/NMC': 4761.9047619047615,
 'BTC/NVC': 2222.222222222222,
 'BTC/PPC': 4545.454545454545,
 'BTC/RUR': 402969.84415,
 'BTC/USD': 7089.998,
 'BTC/ZEC': 38.61003861003861,
 'BTCET/BTC': 0.721,
 'DASH/BCH': 0.4710315591144606,
 'DASH/BTC': 0.04352,
 'DASH/DSHET': 1.3123359580052494,
 'DASH/ETH': 0.784,
 'DASH/EUR': 254.759,
 'DASH/LTC': 2.603,
 'DASH/RUR': 17621.13,
 'DASH/USD': 309.385,
 'DASH/ZEC': 1.688,
 'DSHET/DASH': 0.762,
 'ETH/BCH': 0.6009615384615384,
 'ETH/BTC': 0.05579,
 'ETH/DASH': 1.2755102040816326,
 'ETH/ETHET': 1.4858841010401187,
 'ETH/EUR': 325.4135,
 'ETH/LTC': 3.31205,


In [13]:
'''
Для bid/ask

'''


data_for_graph = get_data_for_graph(price1='bid', price2='ask')
graph = create_graph(data_for_graph)
for key in graph:
    
    path = bellman_ford(graph, key)

    if path not in paths and not None and len(path) > 3:
        paths.append(path)

for path in paths:
    if path == None:
        print("No opportunity here :(")
    else:
        print('cycle lenght:', len(path))
        money = 100
        print("Starting with %(money)i in %(currency)s" % {"money": money, "currency": path[0]})           

        for i, value in enumerate(path):
            if i + 1 < len(path):
                start = path[i]
                end = path[i + 1]
                rate = math.exp(graph[start][end])
                money *= rate*0.998
                print("%(start)s to %(end)s at %(rate)f = %(money)f" % {"start": start, "end": end, "rate": rate,
                                                                        "money": money})
print ("\n")



data for:  bid ask


{'BCH/BCHET': 1.2987012987012987,
 'BCH/BTC': 0.0925,
 'BCH/DASH': 2.118,
 'BCH/ETH': 1.655,
 'BCH/EUR': 538.663,
 'BCH/LTC': 5.501,
 'BCH/RUR': 37132.711,
 'BCH/USD': 654.75,
 'BCH/ZEC': 3.57,
 'BCHET/BCH': 0.766,
 'BTC/BCH': 10.764262648008613,
 'BTC/BTCET': 1.3812154696132597,
 'BTC/DASH': 22.841480127912288,
 'BTC/ETH': 17.873100983020553,
 'BTC/EUR': 5808.58899,
 'BTC/LTC': 59.347181008902076,
 'BTC/NMC': 4545.454545454545,
 'BTC/NVC': 2173.913043478261,
 'BTC/PPC': 4347.826086956522,
 'BTC/RUR': 401564.4833,
 'BTC/USD': 7069.0,
 'BTC/ZEC': 38.46153846153846,
 'BTCET/BTC': 0.721,
 'DASH/BCH': 0.4697040864255519,
 'DASH/BTC': 0.0435,
 'DASH/DSHET': 1.3123359580052494,
 'DASH/ETH': 0.781,
 'DASH/EUR': 253.793,
 'DASH/LTC': 2.594,
 'DASH/RUR': 17554.297,
 'DASH/USD': 309.0,
 'DASH/ZEC': 1.681,
 'DSHET/DASH': 0.761,
 'ETH/BCH': 0.5995203836930456,
 'ETH/BTC': 0.05575,
 'ETH/DASH': 1.272264631043257,
 'ETH/ETHET': 1.4858841010401187,
 'ETH/EUR': 324.30791,
 'ETH/LTC': 3.3048,
 'ETH/RUR

In [12]:


data_for_graph = get_data_for_graph(price1='ask', price2='bid')
graph = create_graph(data_for_graph)
for key in graph:
    
    path = bellman_ford(graph, key)

    if path not in paths and not None and len(path) > 3:
        paths.append(path)

for path in paths:
    if path == None:
        print("No opportunity here :(")
    else:
        print('cycle lenght:', len(path))
        money = 100
        print("Starting with %(money)i in %(currency)s" % {"money": money, "currency": path[0]})           

        for i, value in enumerate(path):
            if i + 1 < len(path):
                start = path[i]
                end = path[i + 1]
                rate = math.exp(graph[start][end])
                money *= rate*0.998
                print("%(start)s to %(end)s at %(rate)f = %(money)f" % {"start": start, "end": end, "rate": rate,
                                                                        "money": money})
print ("\n")




data for:  ask bid


{'BCH/BCHET': 1.3054830287206267,
 'BCH/BTC': 0.0929,
 'BCH/DASH': 2.129,
 'BCH/ETH': 1.668,
 'BCH/EUR': 541.348,
 'BCH/LTC': 5.528,
 'BCH/RUR': 37403.512,
 'BCH/USD': 654.75,
 'BCH/ZEC': 3.593,
 'BCHET/BCH': 0.77,
 'BTC/BCH': 10.81081081081081,
 'BTC/BTCET': 1.3869625520110958,
 'BTC/DASH': 22.98850574712644,
 'BTC/ETH': 17.946877243359655,
 'BTC/EUR': 5844.98049,
 'BTC/LTC': 59.665871121718375,
 'BTC/NMC': 4761.9047619047615,
 'BTC/NVC': 2222.222222222222,
 'BTC/PPC': 4545.454545454545,
 'BTC/RUR': 404375.204,
 'BTC/USD': 7089.988,
 'BTC/ZEC': 38.75968992248062,
 'BTCET/BTC': 0.724,
 'DASH/BCH': 0.47214353163361666,
 'DASH/BTC': 0.04378,
 'DASH/DSHET': 1.314060446780552,
 'DASH/ETH': 0.786,
 'DASH/EUR': 255.725,
 'DASH/LTC': 2.61,
 'DASH/RUR': 17687.963,
 'DASH/USD': 309.77,
 'DASH/ZEC': 1.693,
 'DSHET/DASH': 0.762,
 'ETH/BCH': 0.6024096385542169,
 'ETH/BTC': 0.05595,
 'ETH/DASH': 1.2804097311139564,
 'ETH/ETHET': 1.488095238095238,
 'ETH/EUR': 326.5191,
 'ETH/LTC': 3.32159,
 'ETH/RU